In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import bs4 as BeautifulSoup

from main import *

/Users/arnaubarrera/Desktop/MSc Computer Vision/TFM/labeled_plays_NBA/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
# Initialize the Chrome driver
driver = webdriver.Chrome()

# Navigate to the player statistics page
driver.get("https://www.nba.com/stats/players/traditional")

# Explicit wait to ensure the table is loaded
wait = WebDriverWait(driver, 10)

general_players_table_xpath = '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[3]/table/'

accept_cookies(driver)
time.sleep(3)

# Clicar en cierta columna (tipo de jugada) de un jugador en concreto
try:
    column_indices = {"FGA": 11, "3PA": 14, "REB": 21, "AST": 22, "TOV": 23, "STL": 24, "BLK": 25}

    link_xpath = general_players_table_xpath + f'tbody/tr[1]/td[{column_indices["FGA"]}]/a'
    link_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, link_xpath))
    )

    link_url = link_element.get_attribute("href")
    print(f"Link found: {link_element.get_attribute('href')}")
    driver.get(link_url)

except Exception as e:
    print(f"Error: {e}")


# Create the data frame for the labels of the plays
header_xpath = '//*[@id="__next"]/div[2]/div[2]/div[3]/section/div/div/div[3]/table/thead/tr/th'
header_elements = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.XPATH, header_xpath))
)

# Extraer los nombres de las columnas (headers)
headers = [header.text for header in header_elements]
headers.append("AST")  # Añadir la columna para el link del video
headers.append("Video Link")  # Añadir la columna para el link del video

# Crear el DataFrame vacío con los headers
labels_df = pd.DataFrame(columns=headers)

# Click on some row of the table of plays
video_display_xpath = '//*[@id="vjs_video_3_html5_api"]'

# Load all the rows in one page before iterating
dropdown_xpath = '//*[@class="DropDown_select__4pIg9"]'
option_value = '-1'

dropdown_element = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, dropdown_xpath))
)
select = Select(dropdown_element)
select.select_by_value(option_value)


for i in range(1, 652):  # Iterate over all the plays of a particular player
    line_play = f'//*[@id="__next"]/div[2]/div[2]/div[3]/section/div/div/div[3]/table/tbody/tr[{i}]/td'

    play = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, line_play))
    )

    columns_play = play.find_elements(By.XPATH, line_play)
    
    row_data = []
    for col in columns_play:
        row_data.append(col.text)
    
    play.click()

    # Verify if the shot is assisted
    ast_xpath = '//*[@id="__next"]/div[2]/div[2]/div[3]/section/div/main/section[1]/div/div[2]/h2'

    ast_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, ast_xpath))
    )
    ast_present = 'AST' in ast_element.text
    row_data.append(ast_present)

    # Video display element
    video_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, video_display_xpath))
    )
    video_src = video_element.get_attribute('src')
    row_data.append(video_src)  

    # La fila se agrega al DataFrame
    labels_df.loc[i - 1] = row_data  


driver.quit()

Cookies banner closed.
Link found: https://www.nba.com/stats/events/?CFID=&CFPARAMS=&ContextMeasure=FGA&GameID=&PlayerID=203507&Season=2024-25&SeasonType=Regular%20Season&TeamID=1610612749&flag=3&sct=plot&section=game


In [8]:
# Usando iloc para acceder a la fila 542 (recordando que el índice empieza desde 0, entonces fila 542 será en la posición 541)
video_link = labels_df.iloc[541]['Video Link']
print(video_link)

https://videos.nba.com/nba/static/missing.mp4


In [9]:
def clean_missing_video_data(df: pd.DataFrame):
    """
    Elimina las filas que tienen el vídeo missing.
    """
    # Filtrar las filas donde 'Video Link' no es igual a la URL de video faltante
    df_cleaned = df[df['Video Link'] != 'https://videos.nba.com/nba/static/missing.mp4']
    
    return df_cleaned

In [11]:
labels_df = clean_missing_video_data(labels_df)

In [13]:
import os
import requests
import concurrent.futures
import pandas as pd

def download_video(url: str, output_dir: str):
    """
    Descarga un video desde un URL y lo guarda en el directorio especificado.
    """
    try:
        # Obtener el nombre del archivo a partir de la URL
        video_name = url.split("/")[-1]
        video_path = os.path.join(output_dir, video_name)
        
        # Realizar la solicitud HTTP para descargar el video
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Asegúrate de que la solicitud fue exitosa
        
        # Escribir el video en el archivo local
        with open(video_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)
        print(f"Video descargado: {video_name}")
    
    except requests.exceptions.RequestException as e:
        print(f"Error descargando el video {url}: {e}")


def download_videos_from_df(df: pd.DataFrame, output_dir: str):
    """
    Dado un DataFrame, descarga todos los videos listados en la columna 'Video Link'.
    Los vídeos se guardarán en el directorio 'output_dir'.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)  # Crear el directorio si no existe
    
    # Lista de URLs de los vídeos
    video_links = df['Video Link'].dropna().tolist()
    
    # Usamos concurrent.futures para descargar los vídeos en paralelo
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(download_video, url, output_dir) for url in video_links]
        # Esperar a que todas las tareas de descarga se completen
        for future in concurrent.futures.as_completed(futures):
            future.result()  # Captura cualquier excepción que haya ocurrido

In [ ]:
# Supongamos que tu DataFrame es df
output_dir = "/Users/arnaubarrera/Desktop/MSc Computer Vision/TFM/labeled_plays_NBA/holy_grail"  # Cambia esta ruta por la que desees

# Llamamos a la función para descargar todos los vídeos
download_videos_from_df(labels_df, output_dir)

Video descargado: f2c052bd-e81f-ba18-8c18-66418ef1548d_1280x720.mp4
Video descargado: 7ac52325-8749-0f24-fee9-c90ad6c85274_1280x720.mp4
Video descargado: 2a60554c-1ae1-1b3f-c915-9ffa98bead41_1280x720.mp4
Video descargado: 398e4900-e5da-e058-111f-a08e79ba8b84_1280x720.mp4
Video descargado: 398ef108-31a0-3d77-4a43-357adcd76a85_1280x720.mp4
Video descargado: 24b20d9c-897f-d9d2-892d-8f05a363431c_1280x720.mp4
Video descargado: c23057fd-007a-51d7-80d1-5fc1f8dcb1bf_1280x720.mp4
Video descargado: 4ef7911f-9d8d-2f9d-8abe-2507e9f3c3a8_1280x720.mp4
Video descargado: 660cd839-4216-b35d-e7d6-91b00be40ab4_1280x720.mp4
Video descargado: 0cf4f1de-1c4b-4bc7-87d1-7a00b11b7888_1280x720.mp4
Video descargado: d177c2bf-6553-8873-972b-40ed7c96113d_1280x720.mp4
Video descargado: acc299df-1180-77ca-92be-c70f070c631d_1280x720.mp4
Video descargado: 9d7b0d4b-2044-fdce-5e4d-892afb4055cf_1280x720.mp4
Video descargado: 67253730-2401-74c3-975f-f0047e99ad8e_1280x720.mp4
Video descargado: 29485de8-d6a2-3b67-c25c-000f10

In [6]:
import os

# Define the directory path
output_dir = "/Users/arnaubarrera/Desktop/MSc Computer Vision/TFM/labeled_plays_NBA/holy_grail"

# Initialize counters for files and total size
file_count = 0
total_size = 0

# Walk through the directory
for root, dirs, files in os.walk(output_dir):
    for file in files:
        # Increment the file counter
        file_count += 1
        # Get the size of the file
        file_path = os.path.join(root, file)
        total_size += os.path.getsize(file_path)

# Convert total size from bytes to megabytes
total_size_mb = total_size / (1024 * 1024)

print(f"Total number of files: {file_count}")
print(f"Total size: {total_size_mb:.2f} MB")

Total number of files: 107
Total size: 784.00 MB


In [4]:
import ffmpeg
import os
from tqdm import tqdm

# Ruta de los archivos de vídeo (ajusta esta ruta a la carpeta con tus vídeos MP4)
input_dir = "/Users/arnaubarrera/Desktop/MSc Computer Vision/TFM/labeled_plays_NBA/holy_grail"


# Función para eliminar el audio de un vídeo MP4
def remove_audio(input_video_path, output_video_path):
    try:
        ffmpeg.input(input_video_path).output(output_video_path, an=None).run()  # an=None elimina el audio
        print(f"Audio removed from {input_video_path}")
    except Exception as e:
        print(f"Error processing {input_video_path}: {e}")

# Función para obtener el tamaño total de los archivos en una carpeta
def get_folder_size(folder_path):
    total_size = 0
    total_files = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for file in filenames:
            file_path = os.path.join(dirpath, file)
            total_size += os.path.getsize(file_path)
            total_files += 1
    return total_files, total_size

# Directorio de salida para los vídeos sin audio
output_dir = "/Users/arnaubarrera/Desktop/MSc Computer Vision/TFM/labeled_plays_NBA/no_audio"

# Asegurarse de que el directorio de salida exista
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Obtenemos el tamaño inicial de la carpeta
initial_files, initial_size = get_folder_size(input_dir)
print(f"Initial size: {initial_size / (1024 * 1024):.2f} MB for {initial_files} files")

# Eliminar audio de los vídeos y calcular el tamaño
processed_files = 0
for file in tqdm(os.listdir(input_dir), desc="Processing videos"):
    if file.endswith(".mp4"):
        input_video_path = os.path.join(input_dir, file)
        output_video_path = os.path.join(output_dir, file)
        
        remove_audio(input_video_path, output_video_path)
        processed_files += 1

# Obtenemos el tamaño final de la carpeta de salida
final_files, final_size = get_folder_size(output_dir)
print(f"Final size after removing audio: {final_size / (1024 * 1024):.2f} MB for {final_files} files")

# Muestra la reducción en el tamaño
size_reduction = initial_size - final_size
print(f"Size reduction: {size_reduction / (1024 * 1024):.2f} MB ({(size_reduction / initial_size) * 100:.2f}% reduction)")

Initial size: 784.00 MB for 107 files


Processing videos:   0%|          | 0/107 [00:00<?, ?it/s]ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1_4 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-li

Audio removed from /Users/arnaubarrera/Desktop/MSc Computer Vision/TFM/labeled_plays_NBA/holy_grail/d5467f9f-ccd4-7680-d562-866fd2b85222_1280x720.mp4


[out#0/mp4 @ 0x15280ea60] video:4978KiB audio:0KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.163307%
frame=  685 fps=211 q=-1.0 Lsize=    4986KiB time=00:00:11.38 bitrate=3588.0kbits/s speed= 3.5x    
[libx264 @ 0x152812bc0] frame I:3     Avg QP:23.24  size:139295
[libx264 @ 0x152812bc0] frame P:282   Avg QP:25.93  size: 13769
[libx264 @ 0x152812bc0] frame B:400   Avg QP:31.83  size:  1989
[libx264 @ 0x152812bc0] consecutive B-frames: 19.0%  4.1% 16.2% 60.7%
[libx264 @ 0x152812bc0] mb I  I16..4:  4.1% 40.0% 55.9%
[libx264 @ 0x152812bc0] mb P  I16..4:  0.4%  0.9%  0.6%  P16..4: 30.6% 11.0%  7.0%  0.0%  0.0%    skip:49.5%
[libx264 @ 0x152812bc0] mb B  I16..4:  0.0%  0.1%  0.0%  B16..8: 24.6%  1.7%  0.4%  direct: 0.4%  skip:72.8%  L0:36.0% L1:58.2% BI: 5.8%
[libx264 @ 0x152812bc0] 8x8 transform intra:44.3% inter:49.7%
[libx264 @ 0x152812bc0] coded y,uvDC,uvAC intra: 70.3% 86.5% 58.7% inter: 8.2% 8.5% 1.6%
[libx264 @ 0x152812bc0] i16 v,h,dc,p: 16% 51% 21% 12%


Audio removed from /Users/arnaubarrera/Desktop/MSc Computer Vision/TFM/labeled_plays_NBA/holy_grail/398e4900-e5da-e058-111f-a08e79ba8b84_1280x720.mp4


...

[out#0/mp4 @ 0x14300e580] video:3153KiB audio:0KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.176403%
frame=  397 fps=181 q=-1.0 Lsize=    3159KiB time=00:00:06.58 bitrate=3930.7kbits/s speed=   3x    
[libx264 @ 0x143010a70] frame I:2     Avg QP:24.32  size:118308
[libx264 @ 0x143010a70] frame P:104   Avg QP:26.45  size: 22344
[libx264 @ 0x143010a70] frame B:291   Avg QP:32.12  size:  2295
[libx264 @ 0x143010a70] consecutive B-frames:  1.8%  0.5%  3.0% 94.7%
[libx264 @ 0x143010a70] mb I  I16..4:  1.7% 45.7% 52.5%
[libx264 @ 0x143010a70] mb P  I16..4:  0.3%  2.1%  1.8%  P16..4: 43.7% 18.6% 10.3%  0.0%  0.0%    skip:23.2%
[libx264 @ 0x143010a70] mb B  I16..4:  0.0%  0.0%  0.0%  B16..8: 26.0%  2.2%  0.6%  direct: 0.4%  skip:70.7%  L0:40.1% L1:54.6% BI: 5.2%
[libx264 @ 0x143010a70] 8x8 transform intra:47.9% inter:55.2%
[libx264 @ 0x143010a70] coded y,uvDC,uvAC intra: 82.7% 81.4% 53.9% inter: 9.6% 8.2% 1.0%
[libx264 @ 0x143010a70] i16 v,h,dc,p: 17% 52% 11% 20%


Audio removed from /Users/arnaubarrera/Desktop/MSc Computer Vision/TFM/labeled_plays_NBA/holy_grail/d2b3f276-2c34-01f5-a577-4d81780cbac0_1280x720.mp4


ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1_4 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --e

KeyboardInterrupt: 

[out#0/mp4 @ 0x15772fca0] video:4912KiB audio:0KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.143056%
frame=  519 fps=167 q=-1.0 Lsize=    4919KiB time=00:00:08.61 bitrate=4676.9kbits/s speed=2.77x    
[libx264 @ 0x15772e6b0] frame I:3     Avg QP:24.79  size:112376
[libx264 @ 0x15772e6b0] frame P:132   Avg QP:26.76  size: 27727
[libx264 @ 0x15772e6b0] frame B:384   Avg QP:32.79  size:  2688
[libx264 @ 0x15772e6b0] consecutive B-frames:  1.0%  0.8%  1.2% 97.1%
[libx264 @ 0x15772e6b0] mb I  I16..4:  1.8% 46.8% 51.4%
[libx264 @ 0x15772e6b0] mb P  I16..4:  0.3%  2.2%  2.4%  P16..4: 45.6% 20.1% 11.3%  0.0%  0.0%    skip:18.1%
[libx264 @ 0x15772e6b0] mb B  I16..4:  0.0%  0.0%  0.1%  B16..8: 24.4%  2.4%  0.6%  direct: 0.5%  skip:71.9%  L0:46.8% L1:46.3% BI: 6.9%
[libx264 @ 0x15772e6b0] 8x8 transform intra:44.8% inter:49.4%
[libx264 @ 0x15772e6b0] coded y,uvDC,uvAC intra: 87.8% 89.3% 60.0% inter: 10.8% 9.0% 1.3%
[libx264 @ 0x15772e6b0] i16 v,h,dc,p: 14% 48% 17% 21%